# Predicting loan default with H2o


Orginal notebook link: https://github.com/kguruswamy/H2O3-Driverless-AI-Code-Examples/blob/master/Lending%20Club%20Data%20-%20H2O3%20Auto%20ML%20-%20Python%20Tutorial.ipynb

In [7]:
!pip install bentoml
!pip install h2o xlrd

    100% |████████████████████████████████| 112kB 2.5MB/s ta 0:00:01


In [3]:
import h2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: java version "9.0.1"; Java(TM) SE Runtime Environment (build 9.0.1+11); Java HotSpot(TM) 64-Bit Server VM (build 9.0.1+11, mixed mode)
  Starting server from /usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpggcf7kxv
  JVM stdout: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpggcf7kxv/h2o_bozhaoyu_started_from_python.out
  JVM stderr: /var/folders/kn/xnc9k74x03567n1mx2tfqnpr0000gn/T/tmpggcf7kxv/h2o_bozhaoyu_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,02 secs
H2O cluster timezone:,America/Los_Angeles
H2O data parsing timezone:,UTC
H2O cluster version:,3.24.0.2
H2O cluster version age:,3 months and 8 days
H2O cluster name:,H2O_from_python_bozhaoyu_av60r8
H2O cluster total nodes:,1
H2O cluster free memory:,4 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


## Download the data

In [4]:
import numpy as np
import pandas as pd

import requests
import math
from sklearn import model_selection

In [6]:
!curl -O https://resources.lendingclub.com/LoanStats3c.csv.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 37.1M    0 37.1M    0     0   888k      0 --:--:--  0:00:42 --:--:--  634k-:--:--  941k914k


## Process data

In [8]:
pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth',9999)
pd.set_option('display.max_columns',9999)
pd.set_option('display.max_rows',9999)
data_dictionary = pd.read_excel("https://resources.lendingclub.com/LCDataDictionary.xlsx")

data_dictionary

,LoanStatNew,Description
0,acc_now_delinq,The number of accounts on which the borrower is now delinquent.
1,acc_open_past_24mths,Number of trades opened in past 24 months.
2,addr_state,The state provided by the borrower in the loan application
3,all_util,Balance to credit limit on all trades
4,annual_inc,The self-reported annual income provided by the borrower during registration.
5,annual_inc_joint,The combined self-reported annual income provided by the co-borrowers during registration
6,application_type,Indicates whether the loan is an individual application or a joint application with two co-borrowers
7,avg_cur_bal,Average current balance of all accounts
8,bc_open_to_buy,Total open to buy on revolving bankcards.
9,bc_util,Ratio of total current balance to high credit/credit limit for all bankcard accounts.


In [9]:
# Very first row has non-header data and hence skipping it. Read to a data frame
# Fix the Mon-Year on one column to be readable

def parse_dates(x):
    return datetime.strptime(x, "%b-%d")

lc = pd.read_csv("LoanStats3c.csv.zip", skiprows=1,verbose=False, parse_dates=['issue_d'],low_memory=False) 
lc.shape

(235631, 144)

In [10]:
lc.loan_status.unique()

array(['Charged Off', 'Fully Paid', 'Current', 'In Grace Period',
       'Late (31-120 days)', 'Late (16-30 days)', nan], dtype=object)

In [11]:
# Keep just "Fully Paid" and "Charged Off" to make it a simple 'Yes' or 'No' - binary classification problem

lc = lc[lc.loan_status.isin(['Fully Paid','Charged Off'])]
lc.loan_status.unique()

array(['Charged Off', 'Fully Paid'], dtype=object)

In [12]:
# Drop the columns from the data frame that are Target Leakage ones
# Target Leakage columns are generally created in hindsight by analysts/data engineers/operations after an outcome 
# was detected in historical data. If we don't remove them now, they would climb to the top of the feature list after a model is built and 
# falsely increase the accuracy to 95% :) 
#
# In Production or real life scoring environment, don't expect these columns to be available at scoring time
# , that is,when someone applies for a loan. So we don't train on those columns ...

ignored_cols = [ 
                'out_prncp',                 # Remaining outstanding principal for total amount funded
                'out_prncp_inv',             # Remaining outstanding principal for portion of total amount 
                                             # funded by investors
                'total_pymnt',               # Payments received to date for total amount funded
                'total_pymnt_inv',           # Payments received to date for portion of total amount 
                                             # funded by investors
                'total_rec_prncp',           # Principal received to date 
                'total_rec_int',             # Interest received to date
                'total_rec_late_fee',        # Late fees received to date
                'recoveries',                # post charge off gross recovery
                'collection_recovery_fee',   # post charge off collection fee
                'last_pymnt_d',              # Last month payment was received
                'last_pymnt_amnt',           # Last total payment amount received
                'next_pymnt_d',              # Next scheduled payment date
                'last_credit_pull_d',        # The most recent month LC pulled credit for this loan
                'settlement_term',           # The number of months that the borrower will be on the settlement plan
                'settlement_date',           # The date that the borrower agrees to the settlement plan
                'settlement_amount',         # The loan amount that the borrower has agreed to settle for
                'settlement_percentage',     # The settlement amount as a percentage of the payoff balance amount on the loan
                'settlement_status',         # The status of the borrower’s settlement plan. Possible values are: 
                                             # COMPLETE, ACTIVE, BROKEN, CANCELLED, DENIED, DRAF
                'debt_settlement_flag',      # Flags whether or not the borrower, who has charged-off, is working with 
                                             # a debt-settlement company.
                'debt_settlement_flag_date'  # The most recent date that the Debt_Settlement_Flag has been set
                ]

lc = lc.drop(columns=ignored_cols, axis = 1)

In [13]:
# After dropping Target Leakage columns, we have 223K rows and 125 columns
lc.shape

(227388, 124)

In [14]:
import csv
import os 

train_path = os.getcwd() + "/train_lc.csv.zip"
test_path = os.getcwd() + "/test_lc.csv.zip"

train_lc, test_lc = model_selection.train_test_split(lc, test_size=0.2, random_state=10,stratify=lc['loan_status'])
train_lc.to_csv(train_path, index=False,compression="zip")
test_lc.to_csv(test_path, index=False,compression="zip")



In [15]:
# These two CSV files were created in the previous section

train_path = os.getcwd()+"/train_lc.csv.zip"
test_path = os.getcwd()+ "/test_lc.csv.zip"

train = h2o.load_dataset(train_path)
test = h2o.load_dataset(test_path)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [16]:
train.describe()

Rows:181910
Cols:124




,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount
type,int,int,int,int,int,enum,real,real,enum,enum,enum,enum,enum,real,enum,time,enum,enum,int,string,enum,enum,enum,enum,real,int,time,int,int,int,int,int,int,real,int,enum,int,int,int,enum,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,real,real,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,int,enum,enum,enum,enum,int,real,time,time,time,int,int,enum,real,real,real
mins,NaN,NaN,1000.0,1000.0,950.0,,0.06,23.36,,,,,,3000.0,,1388534400000.0,,,NaN,NaN,,,,,0.0,0.0,-694310400000.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,,0.0,0.0,1.0,,NaN,NaN,NaN,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,16.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,3.0,1.47,1485907200000.0,1491004800000.0,1485907200000.0,3.0,0.0,,4.41,174.15,0.04
mean,0.0,0.0,14681.04460997197,14681.04460997197,14676.151393546239,,0.13707337199714142,440.81824314221376,,,,,,74662.28550865798,,1403525922805.7837,,,0.0,NaN,,,,,17.96228503105935,0.3435215216315754,879745789438.7325,0.7626353691385841,33.4739539409666,70.7152899824253,11.638332142268185,0.2246825353196638,16297.721615084482,0.5542235098700294,25.99993403331318,,0.015716563135616517,42.407435361511055,1.0,,0.0,0.0,0.0,0.005684129514595131,282.4772085097021,138904.83868396413,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,30516.58671870706,0.0,0.0,0.0,4.421785498323331,13344.155553723125,8479.566918329181,64.38059141399033,0.010686603265350997,11.28477818701556,128.2446702925878,185.09749876312512,13.000835578033131,7.947133197735129,1.846737397614203,24.291661578427366,39.74890671309257,6.894262141270328,35.58182123655919,0.5082128525094832,3.6683634764443944,5.779006101918529,4.635786927601561,8.549733384640735,8.56761035676985,8.256736847891812,15.297482271452937,5.74335

## Run H2o on training data set

In [19]:
from h2o.automl import H2OAutoML

# Identify predictors and response
x = train.columns
y = "loan_status"
x.remove(y)


# For binary classification, response should be a factor
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

# Run AutoML 
aml = H2OAutoML(project_name='LC', 
                max_models=50,         # 50 base models
                balance_classes=True,  # Doing smart Class imbalance sampling
                max_runtime_secs=3600, # 1 hours
                seed=1234)             # Set a seed for reproducability
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


## Look at H2o automl leaderboard

In [20]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows)  # Print all rows instead of default (10 rows)

model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20190725_120143,0.719139,0.43074,0.485416,0.367918,0.135364
StackedEnsemble_BestOfFamily_AutoML_20190725_120143,0.718824,0.431016,0.489633,0.36803,0.135446
XGBoost_1_AutoML_20190725_120143,0.716949,0.42889,0.48928,0.367507,0.135061
XGBoost_3_AutoML_20190725_120143,0.714129,0.430712,0.493629,0.368171,0.13555
GLM_grid_1_AutoML_20190725_120143_model_1,0.713017,0.431045,0.493366,0.368307,0.13565
XGBoost_2_AutoML_20190725_120143,0.711692,0.431442,0.493776,0.368656,0.135907


## Test prediction with testing data set

In [70]:
test_pc = aml.predict(test)

stackedensemble prediction progress: |████████████████████████████████████| 100%


/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'emp_title' has levels not trained on: [ Director, Special Projects, #2 process operator, #2process operator, 'roduction manager, (Lead) Asst. Animal Tech., 0perator, 1 unracker lead. 2 Direct care.763-, 100% Owner, 15T UH-60 Repairer, 1LT, 1SG, 1ST ASST MANAGER, 1st Shift Supervisor parts dept., 2ND SHIFT SUPERVISOR, 2nd Assistant A, 2nd Class Warehouseman, 3 rail Elect., 3D Editor, 3D Producer, 3D Project Leader, 3d Artist, 3rd Party A/R, 3rd hand, 4th Grade Teacher , 5 operator, 5th Hand #3 Paper Machine, 600 JACKSON STREET, 6th Grade Social Studies Teacher , 6th grade Math Teacher, 731 union laborer , 8 ft.clipper operator, 9-1-1 Supervisor, 9-1-1 emergency dispatcher , 911 Call Dispatcher, 911 OPERATOR, 911 Specialist/engineer, 911 disoatcher, 911 dispatcher, 92g, A CLASS LINEMAN, A&P mechanic, A-2 Associate, A.h.r, A/ DACG , A/C Technician, A/P Lead, A/R Associ

/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/h2o/job.py:69: UserWarning: Test/Validation dataset column 'title' has levels not trained on: [#2, A#1payoff, AMEX DISC CONSOLIDATION, All Debts Consolidation, AmExPayoffLoan, Auto Credit Card, BILLPAY, Basement Restoration, Better My Home, Bill Me Later/PayPal payoff, Bill's, Bills and Vacation, Bottom line is, Breath easier again, Business expansion for new contract, Business loan, Bussiness1, CC & Student Loan Expenses Refi, CC Consolidate, CC Consolidation Loan, CC Debt Consolidation, CC Finance, CC consolidate, CC consolidation , CCDEBT, CC_Consolidation, CONSOLIDATION , CONSOLIDATIONLOAN, CONSOLODATION, CREDIT CARD PAY OFF, CREDIT CARDS PAYMENT , Car Financing, Car and Home, Card, Card Payoff, Card to find, Cash Flow, Chevrolet, Classic Car Loan, Clearing Credit Cards, Club, Comeback, Condo Improvements, Consodalation, Consoladation , Consolidate CC and close accts, Consolidate Credit Card Balances, Consolidate Creditc

In [71]:
test_pc

predict,Charged Off,Fully Paid
Fully Paid,0.0825953,0.917405
Fully Paid,0.207137,0.792863
Fully Paid,0.131106,0.868894
Fully Paid,0.245828,0.754172
Fully Paid,0.288127,0.711873
Fully Paid,0.105959,0.894041
Fully Paid,0.172084,0.827916
Fully Paid,0.184624,0.815376
Fully Paid,0.0742192,0.925781
Fully Paid,0.123628,0.876372


## Create Machine learning service with BentoML

In [114]:
%%writefile loan_prediction.py

from bentoml import api, env, artifacts, BentoService
from bentoml.artifact import H2oModelArtifact
from bentoml.handlers import DataframeHandler

import h2o

@env(conda_pip_dependencies = ['h2o', 'pandas'])
@artifacts([H2oModelArtifact('model')])
class LoanPrediction(BentoService):
    @api(DataframeHandler)
    def predict(self, df):
        h2o_frame = h2o.H2OFrame(df)
        return self.artifacts.model.predict(h2o_frame)

Overwriting loan_prediction.py


In [118]:
from loan_prediction import LoanPrediction

# Saving the best model for prediction
svc = LoanPrediction.pack(model=aml.leader)
saved_path = svc.save('/tmp/bentoml_archive')
print(saved_path)

[2019-07-25 14:50:04,933] INFO - Searching for dependant modules of loan_prediction:/Users/bozhaoyu/src/bento_gallery/h2o/loan_prediction.py
[2019-07-25 14:50:19,487] INFO - Copying local python module '/Users/bozhaoyu/src/bento_gallery/h2o/loan_prediction.py'
[2019-07-25 14:50:19,493] INFO - Done copying local python dependant modules
[2019-07-25 14:50:19,619] INFO - BentoService LoanPrediction:2019_07_25_12f884ae saved to /tmp/bentoml_archive/LoanPrediction/2019_07_25_12f884ae
/tmp/bentoml_archive/LoanPrediction/2019_07_25_12f884ae


## Using archived ML service from other python application

In [ ]:
from bentoml import load

svc = load(saved_path)

print(svc.predict(test_dataframe))